In [1]:
import pandas as pd
import plotly.express as px

meses = ['JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO']
dados = pd.read_excel(r'FAT DETALHADO 2024 - Bruna Ferreira.xlsx',sheet_name = meses)

lista_dfs = []
for mes in meses:
    df_mes = dados[mes]
    lista_dfs.append(df_mes)

In [2]:
df_agrupado = pd.concat(lista_dfs,ignore_index=True)

In [3]:
df_filtrado = df_agrupado[df_agrupado['Quantidade'] > 0]

In [4]:
df_agrupado.isna().sum()

Cliente                 0
Loja                    0
Nome                    0
N Fantasia              0
Classif                 0
Estado                  0
Municipio               0
Tp Hospital           519
No do Pedido            1
Nota Original           0
Emissao Original        0
Nota Devolucao       2371
Emissao Devolucao    2371
Filial Original         0
Filial Destino       1950
Prefixo                14
Produto                 0
Descricao               0
Lote                    0
Grupo                   0
Quantidade              0
Vlr.Unitario            0
Vlr.Total               0
Desconto                0
Vlr.IPI                 0
Vendedor 1              0
Nome Vend 1             0
Nome do Pac             0
Nome do Hosp          270
Nome Conven.          270
Medico                573
Nome Medico            42
Equipe Med:          2418
Nome da Eq.          2418
Nr. SBOT             1173
Dt Cirurgia           435
dtype: int64

In [5]:
df_agrupado['Nome'].astype('object')
df_agrupado['Nome'].dtypes

dtype('O')

In [6]:
def formatar_brasileiro(valor):
    valor_str = f"{valor:,.2f}"  
    return f"R$ {valor_str.replace(',', 'X').replace('.', ',').replace('X', '.')}"

Principal cliente - cliente potencial - o que mais aparece em faturamento 2024:

1 - maior quantidade de itens faturados naquele cliente;



In [7]:
df_top_clientes = df_filtrado.groupby('Nome')['Quantidade'].count().reset_index().sort_values(by='Quantidade',ascending=False)

fig = px.funnel(df_top_clientes.head(10),x='Quantidade',y='Nome',color='Nome')

fig = fig.update_layout(title={
        "text": "Total de produtos vendidos Por Cliente",
        "x": 0.37,  
        "xanchor": "center",
        "yanchor": "top",
        "font": {"size": 20}  
    }).update_yaxes(showticklabels=False,visible=False,categoryarray=df_top_clientes['Nome'].head(10).to_list())


fig.show()

2 - Maior Faturamento por cliente em 2024

In [8]:
df_top_clientes_vendas = df_filtrado.groupby('Nome')['Vlr.Total'].sum().reset_index().sort_values(by='Vlr.Total',ascending=False)
df_top_clientes_vendas['Vlr.Total'] = df_top_clientes_vendas['Vlr.Total'].astype('int64')

fig = px.funnel(df_top_clientes_vendas.head(10),x='Vlr.Total',y='Nome',color='Nome')
fig = fig.update_layout(title={
        "text": "Faturamento Total Por Clientes",
        "x": 0.37,  
        "xanchor": "center",
        "yanchor": "top",
        "font": {"size": 20}}).update_yaxes(showticklabels=False,visible=False,categoryarray=df_top_clientes_vendas['Nome'].head(10).to_list())


fig.show()


Material foco 2024:

O material mais faturado em 2024 

In [9]:
df_top_produtos = df_filtrado.groupby('Descricao')[['Quantidade','Vlr.Total']].sum().reset_index().sort_values(by='Quantidade',ascending=False)
df_top_produtos['Texto'] = df_top_produtos.apply(
    lambda row: f"Qntd.: {int(row['Quantidade'])}<br>{formatar_brasileiro(row['Vlr.Total'])}", axis=1
)
fig = px.funnel(df_top_produtos.head(10),x=['Quantidade'],y='Descricao',color='Descricao',text='Texto')
fig = fig.update_layout(title={
        "text": "Produto mais Faturado em 2024",
        "x": 0.46,  
        "xanchor": "center",
        "yanchor": "top",
        "font": {"size": 20}},
        margin={"l": 40, "r": 40, "t": 40, "b": 40},  
    height=580,  
    width=1400,  
).update_yaxes(showticklabels=False,visible=False,categoryarray=df_top_produtos['Descricao'].head(10).to_list())

fig.update_traces(
    texttemplate='%{text}',  
    textposition='inside',   
    cliponaxis=False,        
    textfont={"size": 200,"family":"Helvetica Bold"},   
)

fig.show()

In [10]:
lista_cidades = ['XANXERE','SAO MIGUEL DO OESTE','LAGES','CHAPECO','CACADOR','CONCORDIA','JOACABA','VIDEIRA']


df_sul = (df_filtrado[df_filtrado['Municipio'].isin(lista_cidades)])
df_sul = df_sul.groupby('Municipio')['Vlr.Total'].sum().reset_index().sort_values(by='Vlr.Total',ascending=False)
df_sul['Vlr.Total'] = df_sul['Vlr.Total'].astype('int64')
fig = px.funnel(df_sul,x='Vlr.Total',y='Municipio',color='Municipio')
fig = fig.update_layout(title={
        "text": "Faturamento Total Oeste",
        "x": 0.44,  
        "xanchor": "center",
        "yanchor": "top",
        "font": {"size": 20}}).update_yaxes(showticklabels=False,visible=False,categoryarray=df_sul['Municipio'].to_list())


fig.show()